In [ ]:
#Unzipping the model zip files

In [1]:
from timeit import Timer
t = Timer("filepath = '/content/drive/MyDrive/WordEmbedding/e5-small-v2.zip'; \
         extract_to = '/'; \
         from zipfile import ZipFile; \
         ZipFile(filepath).extractall(path=extract_to)")
t.timeit(1)

3.511741959999995

In [7]:

!pip install datasets
!pip install sentence_transformers
from datasets import load_dataset
from sentence_transformers import InputExample
from tqdm.auto import tqdm
from sentence_transformers import datasets,SentenceTransformer


In [9]:
squad_dev = load_dataset('squad_v2', split='validation')

In [8]:
model = SentenceTransformer('/content/e5-small-v2')

In [ ]:
#vec DB

In [10]:
unique_contexts = []
unique_ids = []

# make list of IDs that represent only first instance of
# each context
for row in squad_dev:
    if row['context'] not in unique_contexts:
        unique_contexts.append(row['context'])
        unique_ids.append(row['id'])

# now filter out any samples that aren't included in unique IDs
squad_dev = squad_dev.filter(lambda x: True if x['id'] in unique_ids else False)
squad_dev


Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 1204
})

In [11]:
# and now encode the unique contexts
squad_dev = squad_dev.map(lambda x: {
    'encoding': model.encode(x['context']).tolist()
}, batched=True, batch_size=4)
squad_dev


Map:   0%|          | 0/1204 [00:00<?, ? examples/s]

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers', 'encoding'],
    num_rows: 1204
})

In [12]:
!pip install pinecone-client
import pinecone
API_KEY = '898a7037-44eb-49bb-94a0-95219be317da'

pinecone.init(api_key=API_KEY, environment='gcp-starter')
# (find env next to API key in console


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.4/179.4 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.4/300.4 kB 28.4 MB/s eta 0:00:00


In [13]:
# check if index already exists, if not we create it
if 'squad-index' not in pinecone.list_indexes():
    pinecone.create_index(
        name='squad-index', dimension=model.get_sentence_embedding_dimension(), metric='cosine'
    )


In [14]:
# we use this to get required index dims
model.get_sentence_embedding_dimension()


384

In [15]:
# initialize connection to the new index
index = pinecone.Index('squad-index')


In [16]:
from tqdm.auto import tqdm  # progress bar

upserts = [(v['id'], v['encoding'], {'text': v['context']}) for v in squad_dev]
# now upsert in chunks
for i in tqdm(range(0, len(upserts), 50)):
    i_end = i + 20
    if i_end > len(upserts): i_end = len(upserts)
    index.upsert(vectors=upserts[i:i_end])


  0%|          | 0/25 [00:00<?, ?it/s]

In [18]:
# device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")

In [ ]:
#Query from vec DB

In [19]:
query = "When were the Normans in Normandy?"
xq = model.encode([query]).tolist()


In [ ]:
xc = index.query(xq, top_k=2, include_metadata=True)
xc


In [ ]:
xq = model.encode([
    "How many outputs are expected for each input in a function problem?"
]).tolist()
index.query(xq, top_k=5, include_metadata=True)


In [ ]:
xq = model.encode([
    "Who used Islamic, Lombard, etc construction techniques in the Mediterranean?"
]).tolist()
index.query(xq, top_k=5, include_metadata=True)


In [23]:
xq = model.encode([
    "what is data science?"
]).tolist()
index.query(xq, top_k=5, include_metadata=True)


{'matches': [{'id': '57267b3c5951b619008f7423',
              'metadata': {'text': 'Petrologists can also use fluid inclusion '
                                   'data and perform high temperature and '
                                   'pressure physical experiments to '
                                   'understand the temperatures and pressures '
                                   'at which different mineral phases appear, '
                                   'and how they change through igneous and '
                                   'metamorphic processes. This research can '
                                   'be extrapolated to the field to understand '
                                   'metamorphic processes and the conditions '
                                   'of crystallization of igneous rocks. This '
                                   'work can also help to explain processes '
                                   'that occur within the Earth, such as '
               